# 라이브러리 호출

In [1]:
## 이거는 코랩하시는 분들만 하시면 돼요

#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.stats import gaussian_kde

import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['axes.unicode_minus'] = False #마이너스글자가 깨지는 현상을 해결
#한글 폰트 설정

In [3]:
from google.colab import drive
drive.mount("/content/drive")

ModuleNotFoundError: No module named 'google'

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')
plt.style.use('seaborn')  # matplotlib style 설정
%matplotlib inline

# 데이터 읽기 및 전처리

## Column 설명(0 : 상환 성공, 1 : 상환 실패)
- `'LoanID'` : 각 대출에 대한 고유 식별자
- `'Age'` : 대출자의 나이
- `'Income'`: 대출자의 연봉
- `'LoanAmount'` : 빌린 돈의 액수
- `'CreditScore'` : 대출자의 신용도를 나타내는 신용 점수
- `'MonthsEmployed'` : 대출자가 고용된 달 수
- `'NumCreditLines'` : 대출자가 개설한 신용 라인 수
- `'InterestRate'` : 대출 이자율
- `'LoanTerm'` : 대출 기간(월)
- `'DTIRatio'` : 소득대비 부채를 나타내는 총부채상환비율
- `'Education'` : 대출자가 달성한 최고 수준의 교육
- `'EmploymentType'` : 대출자의 고용형태
- `'MaritalStatus'` : 대출자의 혼인상태
- `'HasMortgage'` : 대출자가 담보대출을 받았는지 여부 (Y/N)
- `'HasDependents'` : 대출자가 부양가족이 있는지 여부 (Y/N)
- `'LoanPurpose'` : 대출의 목적
- `'HasCoSigner'` : 대출에 공동서명자가 있는지 여부 (Y/N)
- `'Default'` : 대출 부도 여부를 나타내는 이진 목표 변수 (0/1)

In [ ]:
df = pd.read_csv("../Downloads/Loan_default.csv")
df

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# Boxplot을 이용한 Outliner 확인

for i in df.describe().columns:
    df[[i]].boxplot()
    plt.show()

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
numeric_data = df.select_dtypes(include=[np.number])  # 숫자형 데이터만 선택
correlation_matrix = numeric_data.corr()  # 숫자형 데이터의 상관관계 행렬 계산

plt.figure(figsize=(18, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn', linewidths=0.2)
plt.show()

## Column 전처리

In [ ]:
useless_col = ['LoanID']
df.drop(useless_col,axis=1,inplace=True)

In [ ]:
df['LoanTerm'] = df['LoanTerm']/12
df.rename(columns={'LoanTerm': 'LoanTerm(year)'}, inplace=True)

In [ ]:
df['Age_Range']=pd.cut(df['Age'],bins=[0,29,39,49,59,69],labels=['10-20대','30대','40대','50대','60대'])

In [ ]:
df["Debt"] = df["DTIRatio"]*df["Income"]

In [ ]:
# LoanAmount_Range 칼럼 생성

df['LoanAmount_Range']=pd.cut(df['LoanAmount'],bins=[5000,66156,127556,188985,249999],labels=['소액','중소액','중고액','고액'])

In [ ]:
# Income_Ragne 칼럼 생성

df['Income_Range']=pd.cut(df['Income'],bins=[0,55572,109520,149999],labels=['저소득층','중소득층','고소득층'])

In [ ]:
# DTI_Range 설정
bins = [0, 0.3, 0.5, 0.7, 0.9]
labels = [1,2,3,4]
df["DTI_Range"] = pd.cut(df["DTIRatio"],bins=bins, labels=labels)

In [ ]:
# Debt_Range 설정
bins = [1500, 17823, 34114, 59301, 135000]
labels = [1,2,3,4]
df["Debt_Range"] = pd.cut(df["Debt"],bins=bins, labels=labels)

In [ ]:
df['Present_Debt'] = (df['Debt']+df['LoanAmount']) # 현재 부채 계산
df['DLIratio'] = df['Present_Debt']/df['Income'] # DLIratio

In [ ]:
df['InterestRate_Year'] = (1 + df['InterestRate']/100)**df['LoanTerm(year)'] # 연간 이자율 계산
df['Total_LoanAmount'] = df['LoanAmount'] * df['InterestRate_Year'] # 최종 납부 금액 계산
df

# EDA

## 전체 데이터셋에서 대해서 비교

In [ ]:
df_fail = df[df['Default']==1]
df_success = df[df['Default']==0]

#### Education


In [ ]:
df['Education'].value_counts()

In [ ]:
df.groupby(['Default','Education']).size().unstack()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# 파이차트
df_fail['Education'].value_counts().plot.pie(
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0], ## 0번째 위치
                                         shadow=True, ## 그림자 표시
                                         fontsize=20)
ax[0].set_title('Each percentage of education of bankrupt')
ax[0].set_ylabel('')

# 카운트 플랏
sns.countplot(x='Education', hue='Default', data=df_fail, ax=ax[1])
ax[1].set_title('Education: bankruptcy or not')

plt.show()

카운트 플롯: 각 학력 별로 전체 대비 파산한 사람의 비율을 보면, 명확하게 차이가 나는 것은 아니지만 High school(12.8%)이 가장 높고 그 다음으로는 Bachelor(12.1%), Master's(10.8%), PhD(10.5%) 였다.  
파이 차트: 파산한 사람들 중 각 학력이 차지하는 비중은 내림차순으로 High school, Bachelor, Master's, PhD 였다.

#### Employment

In [ ]:
df['EmploymentType'].value_counts()

In [ ]:
df.groupby(['Default','EmploymentType']).size().unstack()

In [ ]:
EmploymentType_0 = df.groupby(['Default','EmploymentType']).size().unstack().iloc[0]
EmploymentType_1 = df.groupby(['Default','EmploymentType']).size().unstack().iloc[1]

f,ax=plt.subplots(1,2,figsize=(9,3))

ax[0].pie(EmploymentType_0,labels=EmploymentType_0.index,autopct='%1.1f%%')
ax[0].set_title('Default=0')
ax[1].pie(EmploymentType_1,labels=EmploymentType_1.index,autopct='%1.1f%%')
ax[1].set_title('Default=1')

plt.show()

왼쪽의 파이 차트에서도 확인할 수 있듯이, 대출자에서 고용형태는 4가지 항목이 각각 약 25%로 공평하게 대출이 되었음을 확인할 수 있다.  
하지만 파산한 사람만 골라내어 다시 만든 오른쪽 파이차트에서는 실업자(Unemployed)가 29.2%라는 다소 높은 비율을 차지하고 있음이 확인되었다.  
실업자, 파트 타임, 자영업, 풀타임 근무자 순으로 파산자에서 차지하는 비중이 높다.

#### Marital Status

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# 파이차트1
df['MaritalStatus'].value_counts().plot.pie(
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0],
                                         shadow=True, ## 그림자 표시
                                         fontsize=20)
ax[0].set_title('Overall Marital Status')
ax[0].set_ylabel('')

# 파이차트2
df_fail['MaritalStatus'].value_counts().plot.pie(
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[1],
                                         shadow=True, ## 그림자 표시
                                         fontsize=20)
ax[1].set_title('Marital Status of bankrupt')
ax[1].set_ylabel('')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(5, 5))

# 카운트 플랏
sns.countplot(x='MaritalStatus', hue='Default', data=df)
ax.set_title('Marital Status: bankruptcy or not')
plt.show()

In [ ]:
pd.crosstab(df.Default,
            df.MaritalStatus, margins=True) ## margins=True => 각 행과 열의 총합을 나타내는 "All" 열과 행이 추가

위의 카운트플랏과 크로스대블릿으로 확인한 결과, 각 항목 별로 파산한 비율은 Divorced(12.5%), Married(10.4%), Single(11.9%)순이었다.  
파산한 사람들 중 Divorced가 가장 비중을 많이 차지했고, Divorced 내에서도 파산한 사람의 비율은 12.5%로 가장 높았다.  
하지만 이는 단순히 혼인여부만 따져서는 유의미한 결과를 도출하기 어렵다고 판단하였고, 이에 혼인 여부와 관련된 다른 항목 '부양가족 여부(HasDependents)'와 같이 고려해보기로 하였다.

#### Age

In [ ]:
# Age_Range에 따른 분포
sns.countplot(x='Age_Range', hue='Default', data=df)
plt.title('Age_Range: 성공 vs 실패')
plt.show()

In [ ]:
# 연령대 vs 평균 대출 금액
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='Age_Range', y='LoanAmount', hue='Default', ci=None)

plt.xlabel('연령대')
plt.ylabel('평균 대출 금액')
plt.title('연령대별 평균 대출 금액')
plt.xticks(rotation=45)
plt.legend(title='Default', loc='upper right')
plt.show()

In [ ]:
# 연령대 vs 수입
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='Age_Range', y='Income', hue='Default', ci=None)

plt.xlabel('연령대')
plt.ylabel('수입')
plt.title('연령대별 수입')
plt.xticks(rotation=45)
plt.legend(title='Default', loc='upper right')
plt.show()

In [ ]:
df_fail.columns

In [ ]:
# 'Age_Range'와 'Default' 열을 그룹화하여 대출 부도 건수 계산
age_range_default_count = df_fail.groupby(['Age_Range'])['Default'].count().reset_index()

# 대출 부도 건수를 비율로 변환하여 'Default_Ratio' 열에 저장
total_counts = age_range_default_count['Default'].sum()
age_range_default_count['Default_Ratio'] = age_range_default_count['Default'] / total_counts

# 결과 출력
print(age_range_default_count)


age_range = ['10', '20', '30', '40', '50', '60']
default_ratios = [0.073517, 0.316191, 0.236941, 0.167403, 0.121269, 0.084679]
# 파이 차트 생성
plt.figure(figsize=(4, 4))
plt.pie(default_ratios, labels=age_range, autopct='%1.1f%%', startangle=140)

# 차트 제목 설정
plt.title('Loan default rate by age Range')

# 파이 차트를 원 형태로 유지
plt.axis('equal')

# 차트 표시
plt.show()


- 상환에 실패한 사람들 중 30%이상이 20대에 해당하며, 나이가 많아질수록 부도난 사람이 줄어듦                                                 
- 2030세대가 전체 상환 실패한 사람들 중 55.3%를 차지함

#### Income

In [ ]:
df.describe()

In [ ]:
# Figure 객체와 서브플롯 생성
f, ax = plt.subplots(1, 2, figsize=(20, 10))

# 첫 번째 그래프: 파산한 경우의 수입 분포 히스토그램
df[df['Default'] == 0].Income.plot.hist(ax=ax[0],
                                          bins=20,
                                          edgecolor='black',
                                          color='blue')
ax[0].set_title('Default= 0')  # 제목 설정
x1 = list(range(0,150000, 10000))
ax[0].set_xticks(x1)  # x축 눈금 설정

# 두 번째 그래프: 파산한 경우의 수입 분포 히스토그램
df[df['Default'] == 1].Income.plot.hist(ax=ax[1],
                                          bins=20,
                                          edgecolor='black',
                                          color='blue')
ax[1].set_title('Default= 1')  # 제목 설정
x2 = list(range(0, 150000, 10000))
ax[1].set_xticks(x2)  # x축 눈금 설정
plt.show()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(18, 8))

# 첫 번째 바이올린 플랏
sns.violinplot(x='EmploymentType', y='Income', hue='Default', data=df, split=True)
ax.set_title('Income and EmploymentType vs Default')
ax.set_yticks(range(0, 150000, 10000))

In [ ]:
# Income Histogram
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Income_Range', bins=20, hue='Default', palette='pastel')
plt.xlabel('Income')
plt.ylabel('Frequency')
plt.title('Income Distribution by Default Status')
plt.show()

In [ ]:
# 연령대 vs 수입
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='Income_Range', y='MonthsEmployed', hue='Default', ci=None)

plt.xlabel('수입대')
plt.ylabel('근무개월')
plt.title('수입 vs 근무개월')
plt.xticks(rotation=45)
plt.legend(title='Default', loc='upper right')
plt.show()

In [ ]:
df_low_income = df[df['Income_Range']=='저소득층']
df_mid_income = df[df['Income_Range']=='중소득층']
df_high_income = df[df['Income_Range']=='고소득층']

# 서브플롯 생성
fig, ax = plt.subplots(1, 3, figsize=(18, 8))

# 각 서브플롯에 히스토그램 그리기
sns.histplot(data=df_low_income, x='DTIRatio', bins=20, palette='pastel', ax=ax[0])
ax[0].set_title('DTIRatio - Low Income')

sns.histplot(data=df_mid_income, x='DTIRatio', bins=20, palette='pastel', ax=ax[1])
ax[1].set_title('DTIRatio - Mid-Income')

sns.histplot(data=df_high_income, x='DTIRatio', bins=20, palette='pastel', ax=ax[2])
ax[2].set_title('DTIRatio - High Income')

# 그래프 출력
plt.tight_layout()
plt.show()

#### Loan_Amount

In [ ]:
# Figure 객체와 서브플롯 생성
f, ax = plt.subplots(1, 2, figsize=(20, 10))

# 첫 번째 그래프: 파산한 경우의 수입 분포 히스토그램
df[df['Default'] == 0].LoanAmount.plot.hist(ax=ax[0],
                                          bins=20,
                                          edgecolor='black',
                                          color='blue')
ax[0].set_title('Default= 0')  # 제목 설정
x1 = list(range(0,250000, 10000))
ax[0].set_xticks(x1)  # x축 눈금 설정

# 두 번째 그래프: 파산한 경우의 수입 분포 히스토그램
df[df['Default'] == 1].LoanAmount.plot.hist(ax=ax[1],
                                          bins=20,
                                          edgecolor='black',
                                          color='blue')
ax[1].set_title('Default= 1')  # 제목 설정
x2 = list(range(0, 250000, 10000))
ax[1].set_xticks(x2)  # x축 눈금 설정
plt.show()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(18, 8))

# 첫 번째 바이올린 플랏
sns.violinplot(x='EmploymentType', y='LoanAmount', hue='Default', data=df, split=True)
ax.set_title('EmploymentType and LoanAMount vs Default')
ax.set_yticks(range(0, 250000, 10000))

In [ ]:
# Figure 객체와 서브플롯 생성
f, ax = plt.subplots(1, 2, figsize=(18, 8))

sns.boxplot(x=df['Default'], y=df['LoanAmount'],ax=ax[0])
sns.boxplot(x=df['Default'], y=df['Income'],ax=ax[1])
plt.show()

#### InterestRate

In [ ]:
sns.countplot(x='InterestRate', data=df_fail)
plt.title('InterestRate: 실패')
plt.show()

In [ ]:
df.groupby('Income_Range')['InterestRate'].mean().to_frame()

In [ ]:
# Figure 객체와 서브플롯 생성
f, ax = plt.subplots(1, 2, figsize=(20, 10))

# 첫 번째 그래프: 파산 안한 경우의 이자율 분포 히스토그램
df[df['Default'] == 0].InterestRate.plot.hist(ax=ax[0],
                                          bins=20,
                                          edgecolor='black',
                                          color='blue')
ax[0].set_title('Default= 0')  # 제목 설정
x1 = list(range(0,25, 5))
ax[0].set_xticks(x1)  # x축 눈금 설정

# 두 번째 그래프: 파산한 경우의 수입 분포 히스토그램
df[df['Default'] == 1].InterestRate.plot.hist(ax=ax[1],
                                          bins=20,
                                          edgecolor='black',
                                          color='blue')
ax[1].set_title('Default= 1')  # 제목 설정
x2 = list(range(0, 25, 5))
ax[1].set_xticks(x2)  # x축 눈금 설정
plt.show()

#### 신용도

In [ ]:
plt.hist(x=df.CreditScore, bins=20)
plt.title("CreditScore histogram")
plt.show()

In [ ]:
sns.countplot(x='CreditScore', hue='Default', data=df)
plt.title('신용도 점수: 성공 vs 실패')
plt.show()

In [ ]:
# 신용도 vs 평균 대출 금액
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='CreditScore', y='LoanAmount', ci=None)

plt.xlabel('신용도')
plt.ylabel('대출 금액')
plt.title('신용도별 평균 대출 금액')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Income vs 신용도 점수
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='Income_Range', y='CreditScore', hue='Default', ci=None)

plt.xlabel('수입')
plt.ylabel('신용도 점수')
plt.title('수입대별신용도 점수')
plt.xticks(rotation=45)
plt.legend(title='Default', loc='upper right')
plt.show()

In [ ]:
# Income vs 신용도 점수
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='LoanTerm(year)', y='CreditScore', hue='Default', ci=None)

plt.xlabel('대출기간')
plt.ylabel('신용도 점수')
plt.title('대출기간에 따른 신용도 점수')
plt.xticks(rotation=45)
plt.legend(title='Default', loc='upper right')
plt.show()

In [ ]:
# 연령대 vs 신용도 점수
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='Age_Range', y='CreditScore', hue='Default', ci=None)

plt.xlabel('연령대')
plt.ylabel('신용도 점수')
plt.title('연령대별 신용도 점수')
plt.xticks(rotation=45)
plt.legend(title='Default', loc='upper right')
plt.show()

#### 대출 목적(business)

In [ ]:
df.LoanPurpose.value_counts()

In [ ]:
# 연령대 vs 수입
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='LoanPurpose', y='LoanAmount', hue='Default', ci=None)

plt.xlabel('대출 목적')
plt.ylabel('대출 금액')
plt.title('대출 목적에 따른 대출 금액')
plt.xticks(rotation=45)
plt.legend(title='Default', loc='upper right')
plt.show()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# 파이차트
df_fail['LoanPurpose'].value_counts().plot.pie(
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0], ## 0번째 위치
                                         shadow=True, ## 그림자 표시
                                         fontsize=20)
ax[0].set_title('Each percentage of purpose of bankrupt')
ax[0].set_ylabel('')

# 카운트 플랏
sns.countplot(x='LoanPurpose', hue='Default', data=df, ax=ax[1])
ax[1].set_title('LoanPurpose: bankruptcy or not')

plt.show()

In [ ]:
df.groupby('LoanPurpose')[['InterestRate','LoanAmount']].mean()

In [ ]:
loan_purpose_sum = df.groupby('LoanPurpose')['LoanAmount'].sum()

# 파이 차트 그리기
plt.figure(figsize=(8, 8))
plt.pie(loan_purpose_sum, labels=loan_purpose_sum.index, autopct='%1.1f%%', startangle=140, colors = ['skyblue', 'lightgreen', 'lightcoral', 'lightsalmon', 'lightpink'])
plt.axis('equal')  # 원형을 유지
plt.title('대출 목적에 따른 대출 금액 비율')
plt.show()

In [ ]:
df_business = df[df['LoanPurpose']=='Business']
df_business

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df_business, x='LoanAmount', bins=20, hue='Default', palette='pastel')
plt.xlabel('LoanAmount')
plt.ylabel('Frequency')
plt.title('LoanAmount Distribution by Default Status')
plt.show()

#### DTI 분석

In [ ]:
sns.countplot(x='DTIRatio', hue='Default', data=df)
plt.title('DTiRato: 성공 vs 실패')
plt.show()

#상품 제안

여기서부터 코드 자유롭게 붙여주세요~~~~

1. 저소득자, 저신용자들 위한 상품 제안 (보미)
2. 사업자를 위한 상품 제안 (진호, 수빈)
3. 실업자 장려 대출 정책 (범윤, 지윤)

# 사업자를 위한 대출상품 - 수빈

## 'DSR' column 생성

In [ ]:
# DSR의 분자에 포함되는 원리합계의 현재가치를 먼저 계산하는 함수를 먼저 작성
def future_value(row):
    rate = row['InterestRate'] / 100  # 백분율로 주어진 이자율을 소수로 변환
    term = int(row['LoanTerm(year)'])
    principal = row['LoanAmount']

    total_fv = 0
    for n in range(1, term + 1):
        if n < term:
            # 각 연도의 이자에 대한 현재 가치를 누적
            total_fv += (principal * rate) * (1 + rate) ** n
        else:
            # 마지막 연도의 원금과 이자에 대한 현재 가치를 누적
            total_fv += (principal + principal * rate) * (1 + rate) ** n
            
    return total_fv

In [ ]:
df['FV_Total'] = df.apply(future_value, axis=1)

In [ ]:
#DSR 작성
df['DSR'] = (df['FV_Total'] + df['Debt']) /df['Income']
df.head()

## 'LoanPurpose' 제한

In [ ]:
numeric_data = df.select_dtypes(include=[np.number])  # 숫자형 데이터만 선택
correlation_matrix = numeric_data.corr()  # 숫자형 데이터의 상관관계 행렬 계산

plt.figure(figsize=(18, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn', linewidths=0.2)
plt.show()

In [ ]:
df_bus = df.loc[df['LoanPurpose']=='Business']
df_bus.head()

In [ ]:
# 중간값, 결측치, 왜도, 첨도를 표시합니다.
df_stats = df_bus.describe().T

median_results = []
skew_results = []
kurtosis_results = []
null_results = []

for idx, val in enumerate(df_stats.index):
    median_results.append(df[val].median())
    skew_results.append(df[val].skew())
    kurtosis_results.append(df[val].kurtosis())
    null_results.append(df[val].isnull().sum())

df_stats['median'] = median_results
df_stats['missing'] = null_results
df_stats['skewness'] = skew_results
df_stats['kurtosis'] = kurtosis_results
df_stats

In [ ]:
df_bus.hist(figsize=(22,18), density=True)
plt.show()

In [ ]:
sns.kdeplot(x='InterestRate',data=df_bus,fill=True, label='Target')
sns.kdeplot(x='InterestRate',data=df,fill=True, label='Total')
plt.legend()
plt.show()

## 'Age' 제한

In [ ]:
# Age_Range에 따른 분포
sns.countplot(x='Age_Range', hue='Default', data=df_bus)
plt.title('Age_Range: 성공 vs 실패')
plt.show()

In [ ]:
df_bus.groupby(by='Age_Range')[['Default','Age','Income','LoanAmount','CreditScore','InterestRate','LoanTerm(year)','Debt','DSR']].mean()

In [ ]:
df_bus3060 = df_bus.loc[df_bus['Age_Range']!='10-20대']
df_bus3060['Debt'].mean()

## 'Income' 제한

In [ ]:
# Income_Range에 따른 분포
sns.countplot(x='Income_Range', hue='Default', data=df_bus)
plt.title('Income_Range: 성공 vs 실패')
plt.show()

In [ ]:
df_bus.groupby(by='Income_Range')[['Default','Age','Income','LoanAmount','CreditScore','InterestRate','LoanTerm(year)','Debt','DSR']].mean()

In [ ]:
df_busHigh = df_bus.loc[df_bus['Income_Range']!='저소득층']
df_busHigh['DSR'].mean()

In [ ]:
df_bus.groupby('Income_Range')['DSR'].describe()

In [ ]:
lower_limit = 0
upper_limit = 30
sns.kdeplot(x='DSR',data=df_bus,hue='Income_Range',fill=True, clip=(lower_limit, upper_limit))

## 대상(df_target) 분석

In [ ]:
df_target = df_bus.drop(df_bus[(df_bus['Age_Range']=='10-20대')&(df_bus['Debt']>41269)].index)
df_target = df_target.drop(df_target[(df_target['Income_Range']=='저소득층')&(df_target['DSR']>23.23)].index)
df_target.head()

In [ ]:
df_target.describe()

In [ ]:
df_bus.describe()

In [ ]:
df_target0=df_target.loc[df_target['Default']==0]
df_target1=df_target.loc[df_target['Default']==1]

In [ ]:
f, ax = plt.subplots(2, 6, figsize=(20, 8))

# 파이차트1
df_target0['Education'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0,0],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[0,0].set_title('Education(Default=0)')
ax[0,0].set_ylabel('')

# 파이차트2
df_target0['EmploymentType'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0,1],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[0,1].set_title('EmploymentType(Default=0)')
ax[0,1].set_ylabel('')

#파이차트3
df_target0['MaritalStatus'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0,2],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[0,2].set_title('MaritalStatus(Default=0)')
ax[0,2].set_ylabel('')


#파이차트4
df_target0['HasMortgage'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0,3],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[0,3].set_title('HasMortgage(Default=0)')
ax[0,3].set_ylabel('')

#파이차트5
df_target0['HasDependents'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0,4],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[0,4].set_title('HasDependents(Default=0)')
ax[0,4].set_ylabel('')


#파이차트7
df_target0['HasCoSigner'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[0,5],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[0,5].set_title('HasCoSigner(Default=0)')
ax[0,5].set_ylabel('')


#####################################################


# 파이차트8
df_target1['Education'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[1,0],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[1,0].set_title('Education(Default=1)')
ax[1,0].set_ylabel('')

# 파이차트9
df_target1['EmploymentType'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[1,1],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[1,1].set_title('EmploymentType(Default=1)')
ax[1,1].set_ylabel('')

#파이차트10
df_target1['MaritalStatus'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[1,2],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[1,2].set_title('MaritalStatus(Default=1)')
ax[1,2].set_ylabel('')


#파이차트11
df_target1['HasMortgage'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[1,3],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[1,3].set_title('HasMortgage(Default=1)')
ax[1,3].set_ylabel('')

#파이차트12
df_target1['HasDependents'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[1,4],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[1,4].set_title('HasDependents(Default=1)')
ax[1,4].set_ylabel('')

#파이차트14
df_target1['HasCoSigner'].value_counts().plot.pie( 
                                         autopct='%1.1f%%', ## 백분율을 소수점 앞 한자리 소수점 아래 한자리까지 %로 표시
                                         ax=ax[1,5],
                                         shadow=True, ## 그림자 표시
                                         fontsize=10)
ax[1,5].set_title('HasCoSigner(Default=1)')
ax[1,5].set_ylabel('')

## 'LoanAmount' 상한선 측정

In [ ]:
sns.kdeplot(x='LoanAmount',data=df_target,hue='Default',fill=True)

# 커널 밀도 추정을 수행합니다.
x = np.linspace(min(df_target['LoanAmount']), max(df_target['LoanAmount']), 1000)
kde_default = gaussian_kde(df_target[df_target['Default'] == 1]['LoanAmount'])
kde_not_default = gaussian_kde(df_target[df_target['Default'] == 0]['LoanAmount'])

# 밀도 값을 계산합니다.
y_default = kde_default(x)
y_not_default = kde_not_default(x)

# 최대 밀도 값을 갖는 x 위치를 찾습니다.
x_max_default = x[np.argmax(y_default)]
x_max_not_default = x[np.argmax(y_not_default)]

print(f"Default=Yes의 최대 밀도 x 위치: {x_max_default}")
print(f"Default=No의 최대 밀도 x 위치: {x_max_not_default}")

In [ ]:
df_target[['LoanAmount']].describe()

In [ ]:
df_target['LoanAmount/Income']=df_target['LoanAmount']/df_target['Income']
df_target['LoanAmount/Income'].describe()

In [ ]:
df['LoanAmount/Income']=df['LoanAmount']/df['Income']
df['LoanAmount/Income'].describe()

In [ ]:
sns.kdeplot(x='LoanAmount/Income', data=df_target.loc[df_target['Default']==1], fill=True, palette='viridis',label='Target')
sns.kdeplot(x='LoanAmount/Income', data=df.loc[df['Default']==1], fill=True, palette='pastel',label='Total')
plt.legend()
plt.show()

In [ ]:

# 1. 각 데이터셋에 대한 KDE 값을 추정
x_grid = np.linspace(min(df_target['LoanAmount/Income'].min(), df['LoanAmount/Income'].min()), 
                     max(df_target['LoanAmount/Income'].max(), df['LoanAmount/Income'].max()), 1000)

kde_target = gaussian_kde(df_target.loc[df_target['Default']==1, 'LoanAmount/Income'])
kde_df = gaussian_kde(df.loc[df['Default']==1, 'LoanAmount/Income'])

kde_vals_target = kde_target(x_grid)
kde_vals_df = kde_df(x_grid)

# 2. 두 KDE 값의 차이 계산
diff = kde_vals_target - kde_vals_df

# 3. 교차 지점 찾기
cross_points = x_grid[np.where(np.diff(np.sign(diff)))[0]]

print(cross_points)


## 적정 'LoanTerm(year)' 측정

In [ ]:
sns.histplot(x='LoanTerm(year)',data=df_target.loc[df_target['Default']==1],fill=True,label='Target')
sns.histplot(x='LoanTerm(year)',data=df.loc[df['Default']==1],fill=True,label='Total')
plt.legend()
plt.show()

In [ ]:
sns.kdeplot(x='LoanTerm(year)',data=df_target.loc[df_target['Default']==1],fill=True)
sns.kdeplot(x='LoanTerm(year)',data=df.loc[df['Default']==1],fill=True)
plt.show()

## 적정 'InterestRate' 측정

In [ ]:
sns.kdeplot(x='InterestRate',data=df_target,hue='Default',fill=True)

# 커널 밀도 추정을 수행합니다.
x = np.linspace(min(df_target['InterestRate']), max(df_target['InterestRate']), 1000)
kde_default = gaussian_kde(df_target[df_bus['Default'] == 1]['InterestRate'])
kde_not_default = gaussian_kde(df_target[df_bus['Default'] == 0]['InterestRate'])

# 밀도 값을 계산합니다.
y_default = kde_default(x)
y_not_default = kde_not_default(x)

# 최대 밀도 값을 갖는 x 위치를 찾습니다.
x_max_default = x[np.argmax(y_default)]
x_max_not_default = x[np.argmax(y_not_default)]

print(f"Default=Yes의 최대 밀도 x 위치: {x_max_default}")
print(f"Default=No의 최대 밀도 x 위치: {x_max_not_default}")

In [ ]:
df_target['LoanAmount'].describe()

### 최대 적정 이자율 찾기

In [ ]:
# KDE 플롯 생성
sns.set(style="whitegrid")
sns.kdeplot(x='InterestRate', data=df_target, hue='Default', fill=True, legend=True)

# 그래프 표시
plt.show()

In [ ]:
# KDE 플롯 생성
sns.set(style="whitegrid")
sns.kdeplot(x='InterestRate', data=df_target.loc[df_target['Default']==1], fill=True, palette='viridis',label='Target')
sns.kdeplot(x='InterestRate', data=df.loc[df['Default']==1], fill=True, palette='pastel', label='Total')

# 그래프 표시
plt.legend()
plt.show()

In [ ]:

# 1. 각 데이터셋에 대한 KDE 값을 추정
x_grid = np.linspace(min(df_target['InterestRate'].min(), df['InterestRate'].min()), 
                     max(df_target['InterestRate'].max(), df['InterestRate'].max()), 1000)

kde_target = gaussian_kde(df_target.loc[df_target['Default']==1, 'InterestRate'])
kde_df = gaussian_kde(df.loc[df['Default']==1, 'InterestRate'])

kde_vals_target = kde_target(x_grid)
kde_vals_df = kde_df(x_grid)

# 2. 두 KDE 값의 차이 계산
diff = kde_vals_target - kde_vals_df

# 3. 교차 지점 찾기
cross_points = x_grid[np.where(np.diff(np.sign(diff)))[0]]

print(cross_points)

### 최소 적정 이자율 찾기

In [ ]:
# DSR의 분자에 포함되는 원리합계의 현재가치를 먼저 계산하는 함수를 먼저 작성
def present_new_value(row):
    rate = row['New_InterestRate'] / 100  # 백분율로 주어진 이자율을 소수로 변환
    term = int(row['LoanTerm(year)'])
    principal = row['LoanAmount']

    total_fv = 0

    for n in range(1, term + 1):
        if n < term:
            # 각 연도의 이자에 대한 현재 가치를 누적
            total_fv += (principal * rate) * (1 + rate) ** n
        else:
            # 마지막 연도의 원금과 이자에 대한 현재 가치를 누적
            total_fv += (principal + principal * rate) * (1 + rate) ** n
            
    return total_fv


In [ ]:
df_target_success =  df_target[df_target['Default']==0]
df_target_fail = df_target[df_target['Default']==1]
df_target_success['New_InterestRate']=0
total_revenue_threshold = df_target_fail['LoanAmount'].sum()
current_total_revenue=0

while current_total_revenue <= total_revenue_threshold:
  df_target_success['New_Total_LoanAmount'] = df_target_success.apply(present_new_value,axis=1)
  df_target_success['Total_Revenue'] = df_target_success['New_Total_LoanAmount'] - df_target_success['FV_Total']
  current_total_revenue = df_target_success['Total_Revenue'].sum()
  df_target_success['New_InterestRate'] += 0.1

df_target_success['New_InterestRate']